In [1]:
from typing import List,Optional

from pydantic import BaseModel, Field


class Company(BaseModel):
    """Extracted data about company's news."""

    # Creates a model so that we can extract multiple entities.
    name: Optional[str] = Field(default=None, description="Name of the company")
    ticker: Optional[str] = Field(default=None, description="Ticker symbol of the company")
    content: List[str] = Field(default=None, description="News content related to the company")

class News(BaseModel):
    """Extract the Macro News, and the News bite by each company"""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    macro: List[str] = Field(default=None, description="Macroeconomic news")
    company: List[Company] = Field(
        default=None, description="Extracted data about companies' news."
    )

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)



In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None,
    # reasoning_format="hidden",
    timeout=None,
    max_retries=0,
    # other params...
)

In [5]:
structured_llm = llm.with_structured_output(schema=News)
text = "Review & Outlook\nBursa Malaysia shares ended higher on Thursday, following gains in major regional markets, as recent trade developments between the U.S. and its key partners boosted investor confidence. The FBM KLCI jumped 10.53 points to close at the day's high of 1,540.32, off an early low of 1,529.71, as gainers led losers 502 to 474, on modest trade totaling 2.97bn shares worth RM2.3bn.\nThe local benchmark is poised to extend its gains heading into the weekend, underpinned by improving momentum indicators and renewed optimism over prospective trade agreements with the U.S. Immediate index support is kept at 1,490, with stronger support found at 1,465 followed by 1,444. Immediate resistance remains at 1,564 with next upside hurdles seen at the recent high of 1,586, followed by 1,610 ahead.\nHartalega may have bottomed out and is set for recovery to target the 50-day ma (RM1.70), 76.4%FR (RM1.99) and 61.8%FR (RM2.38) ahead, with key chart supports at RM1.09 and the 123.6%FP (75sen) cushioning downside. Kossan shares are poised for recovery, with a confirmed breakout above the 23.6%FR (RM1.52) and 38.2%FR (RM1.78) to enhance upside momentum towards the 50%FR (RM1.99) ahead, while strong retracement supports are seen at the 08/05/23 low (RM1.11) and 89sen.\nNews Bites\nMalaysia has secured over RM63bn worth of semiconductor investments under its National Semiconductor Strategy as of March 2025, less than a year after its launch in May 2024.\nThe Investment, Trade and Industry Minister Tengku Datuk Seri Zafrul Abdul Aziz denied a Bloomberg report that Malaysia was eyeing a 20% tariff and said Malaysia is still on track to secure a trade deal with the US by the Aug 1 deadline at the lowest possible tariff.\nMMAG Holdings Bhd has entered into a conditional aircraft sale agreement to acquire a Boeing 737-800BCF freighter for US$25.9mn (RM109.9mn) cash.\nKeyfield International Bhd has secured 2 work orders from Petronas Carigali Sdn Bhd for the provision of accommodation work boats to support the national oil company's offshore operations.\nKinergy Advancement Bhd has secured fee-in approvals for two hydropower projects with a total installed capacity of 8.04MW under the Sustainable Energy Development Authority's FiT 2.0 initiative.\nRimbunan Sawit Bhd has announced the termination of its JV with LCDA Holdings Sdn Bhd to develop a parcel of native customary rights land in Ulu Selangau, Sibu into an oil palm plantation.\nJentayu Sustainables Bhd said Sumitomo Corporation does not intend to extend a MoU the parties signed 2 years ago to collaborate on renewable energy projects.\nAYS Ventures Bhd said the voluntary takeover offer by its associate company 3HA Capital Pte Ltd for all shares in Singapore-listed CosmoSteel Holdings Ltd has become unconditional, after 3HA Capital's shareholding crossed the 50% threshold.\nCountry Heights Holdings Bhd's wholly-owned unit, Country Heights Commercial Development Sdn Bhd has been served with a winding-up petition by the Inland Revenue Board over RM312,560 in unpaid taxes.\nMetronic Global Bhd is ending its JV with Zhuhai SingYes New Materials Technology Co Ltd to develop and market smart city solutions across the country.\nKIP Real Estate Investment Trust's FY25 net property income surged 24.4% YoY to RM96.8mn, mainly driven by the contribution from its 7 KIPMalls and its 4 industrial properties, as well as the newly bought D'pulze Shopping Centre and TF Value-Mart.\nLuxchem Corporation Bhd's 2QFY25 net profit declined by 27.1% YoY to RM8.9mn amid lower sales from its business segments.\nBetamek Bhd's 1QFY26 net profit rose 10.5% YoY to RM5.4mn due to consistent favourable performance of the ringgit against the USD and the CNY as well as the better expenses monitoring by each unit of the group.\nThe Philippines aims to bring down the tariff imposed by the US on its exports to 15% from 19%.\nUS President Donald Trump suggested that he would not go below 15% as he sets so-called reciprocal tariff rates ahead of an Aug. 1 deadline.\nInitial claims for state unemployment benefits in the US dropped 4,000 to a seasonally adjusted 217,000 for the week ended July 19, the lowest level since April.\nS&P Global's flash US Composite PMI Output Index increased to 54.6 this month, the highest level since December, from 52.9 in June.\nSource:TA Research- 25 Jul 2025"
prompt = prompt_template.invoke({"text": text})
output = structured_llm.invoke(prompt)



In [ ]:
# Create an agent to validae the output, if no ticker is found, search for it,

In [11]:
output_dict = output.model_dump()
output_dict

{'macro': ['Malaysia has secured over RM63bn worth of semiconductor investments under its National Semiconductor Strategy as of March 2025, less than a year after its launch in May 2024',
  'The Investment, Trade and Industry Minister Tengku Datuk Seri Zafrul Abdul Aziz denied a Bloomberg report that Malaysia was eyeing a 20% tariff and said Malaysia is still on track to secure a trade deal with the US by the Aug 1 deadline at the lowest possible tariff',
  'The Philippines aims to bring down the tariff imposed by the US on its exports to 15% from 19%',
  'US President Donald Trump suggested that he would not go below 15% as he sets so-called reciprocal tariff rates ahead of an Aug. 1 deadline',
  'Initial claims for state unemployment benefits in the US dropped 4,000 to a seasonally adjusted 217,000 for the week ended July 19, the lowest level since April',
  "S&P Global's flash US Composite PMI Output Index increased to 54.6 this month, the highest level since December, from 52.9 in 